In [1]:
import stanza
from chapeau_parse import chapeau_parse
from utils import get_maximal_chapeaus

/Users/bw/Documents/chapeau_parse/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nlp_tok = stanza.Pipeline('en', processors='tokenize')
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,depparse,constituency', tokenize_pretokenized=True)

2024-10-25 00:01:27 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-25 00:01:27 INFO: Downloaded file to /Users/bw/stanza_resources/resources.json
2024-10-25 00:01:27 WARNING: Language en package default expects mwt, which has been added
2024-10-25 00:01:27 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

2024-10-25 00:01:27 INFO: Using device: cpu
2024-10-25 00:01:27 INFO: Loading: tokenize
2024-10-25 00:01:28 INFO: Loading: mwt
2024-10-25 00:01:28 INFO: Done loading processors!
2024-10-25 00:01:28 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-25 00:01:28 INFO: Downloaded fil

In [3]:
with open("usc29.xml", 'r') as file:
        usc29_raw = file.read()

In [4]:
from bs4 import BeautifulSoup
from utils import remove_notes
usc29_xml = BeautifulSoup(usc29_raw, 'xml')
print("Number of elements before removing notes: " + str(len(usc29_xml.find_all())))
usc29_xml = remove_notes(usc29_xml)
print("Number of elements after removing notes: " + str(len(usc29_xml.find_all())))

Number of elements before removing notes: 112711
Number of elements after removing notes: 71218


In [5]:
maximal_chapeaus = get_maximal_chapeaus(usc29_xml)
len(maximal_chapeaus)

489

In [6]:
len(usc29_xml.get_text().split(" "))

548714

In [8]:
import os
import traceback
from tqdm import tqdm
for id, chapeau in tqdm(list(maximal_chapeaus.items())[:50]):
    # create folder with name of id if it doesn't already exist
    dir = 'results/'+id
    if not os.path.exists(dir):
        os.makedirs(dir)
    # try to parse chapeau. if it fails, save the error message to a file and continue
    try:
        parse = chapeau_parse(chapeau, nlp_tok, nlp)
        parse.constparse_to_pdf('{}/{}-constparse.pdf'.format(dir,id))
        parse.depparse_to_pdf('{}/{}-deppparse.pdf'.format(dir,id))
        with open ('{}/{}-tok.txt'.format(dir,id), 'w') as file:
            file.write(parse.tok)
        if parse.continuation_constparses != []:
            parse.continuation_constparses_to_pdf('{}/{}-contconstparse.pdf'.format(dir,id))
        if parse.continuation_depparses != []:
            parse.continuation_depparses_to_pdf('{}/{}-contdeppparse.pdf'.format(dir,id))
    except Exception as e:
        with open('{}/{}-traceback.txt'.format(dir,id), 'w') as file:
            file.write(traceback.format_exc())
        with open('{}/{}-error.txt'.format(dir,id), 'w') as file:
            file.write(str(e))
        continue

100%|██████████| 50/50 [16:59<00:00, 20.39s/it]


In [7]:
# investigate the error in id296b2ce4-2a73-11ef-ba6d-a9961dfb011e
chapeau_parse(maximal_chapeaus['id296b2ce4-2a73-11ef-ba6d-a9961dfb011e'], nlp_tok, nlp)

IndexError: list index out of range